In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from scipy.spatial.distance import cosine
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
train_data = pd.read_csv('train_data.csv')
train_labels = pd.read_csv('train_labels.csv')

In [3]:
train_data = train_data[['id','question1','question2']].merge(train_labels)

In [4]:
train, test = train_test_split(train_data, train_size = 0.8, random_state = 49)

In [5]:
train.head()

,id,question1,question2,is_duplicate
125350,156795,What are some examples of passive-aggressive b...,What is an example of a passive aggressive que...,0
298955,374113,How can I lose weight at age 55?,How do I lose weight without quitting?,1
270303,338301,Who are your countries greatest heroes?,Who are some Jewish heroes?,0
127595,159621,Why do people write for Quora?,Why do you answer questions in Quora?,1
183095,229124,"What does Gradle do in Android Studio, and sho...",What is gradle exactly in android studio?,1


In [6]:
train['is_duplicate'].value_counts()

0    162861
1     95670
Name: is_duplicate, dtype: int64

In [7]:
all_questions = pd.concat([train['question1'],train['question2']]).reset_index().drop('index', axis = 1)

In [8]:
stop = stopwords.words('english') + list(string.punctuation)

def tokenize(sentence):
    tokens = [i for i in nltk.word_tokenize(sentence.lower()) if i not in stop]
    return tokens


In [9]:
all_q = all_questions.values.tolist()

In [10]:
tokenized = []

for i in range(0,len(all_q)):
    tokenized.append(tokenize(str(all_q[i])))

In [11]:
w2v = Word2Vec(size=200, min_count=1)
w2v.build_vocab([x for x in tokenized])
w2v.train([x for x in tokenized],total_examples=w2v.corpus_count,epochs=w2v.iter)

15084925

In [246]:
def average_vector(words):
    return np.divide(np.sum([w2v[x] for x in words],0),np.minimum(len(words),1))

In [261]:


train.dropna()

x1 = []
x2 = []
y = []

In [262]:
x1q = train['question1'].reset_index().drop('index', axis = 1).values.tolist()
for i in range(0,len(train)):
    x1.append(tokenize(str(x1q[i])))

In [263]:
x2q = train['question2'].reset_index().drop('index', axis = 1).values.tolist()
for i in range(0,len(train)):
    x2.append(tokenize(str(x2q[i])))

In [264]:
y = train['is_duplicate'].reset_index().drop('index', axis = 1).values.tolist()

In [265]:
def cosine_sim(sen1,sen2):
    return cosine(average_vector(sen1),average_vector(sen2))

In [266]:
distance = []
y_new = []
for i in range(0,len(train)):
    try:
        distance.append(cosine_sim(x1[i],x2[i]))
        y_new.append(y[i])
    except:
        pass

/Users/tom/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in divide
  


In [267]:
len(distance)

258522

In [268]:
len(y_new)

258522

In [272]:
distance_arr = np.array(distance)
y_arr = np.array(y_new)

In [273]:

logreg = LogisticRegression()
logreg.fit(distance_arr.reshape(-1, 1), y_arr)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [276]:
logreg.score(distance_arr.reshape(-1, 1), y_arr)

0.65561151468733803